In [33]:
import torch
import numpy as np


In [34]:
print(torch.__version__)

2.1.2


In [35]:
t1=torch.tensor(4.0)
t1

tensor(4.)

In [36]:
t1.dtype

torch.float32

In [37]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [38]:
t2.dtype

torch.float32

In [39]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [40]:
t4=torch.tensor([
    [[11,12,13],
     [13,14,15]],
     [[15,16,17],
      [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [41]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [42]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [43]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [44]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [45]:
# bnasic tensor operations
x=torch.tensor(3.0)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [46]:
x.is_leaf

True

In [47]:
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

In [48]:
# y.retain_grad()
y.backward()
print('dy/dx:',x.grad)
print('dy/dw:',w.grad)
print('dy/db:',b.grad)
print('dy/dw 2:',w.grad_fn)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)
dy/dw 2: None


In [49]:
w2=torch.tensor(42.,requires_grad=True)
b2=torch.tensor(52.,requires_grad=True)
y2=w2+b2

In [50]:
y2.backward
print(w2.grad)

None


In [51]:
t6=torch.full((3,2),42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [52]:
t7=torch.concatenate((t3,t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [53]:
t8=torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [54]:
t9=t8.reshape(3,2,2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

In [55]:
x=np.array([[1,2],
            [3,4.]])
x

array([[1., 2.],
       [3., 4.]])

In [56]:
y=torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [57]:
x.dtype,y.dtype

(dtype('float64'), torch.float64)

In [58]:
z=y.numpy()
z

array([[1., 2.],
       [3., 4.]])

Fundamentals of Pytorch and Numpy done.
Next progress towards linear regression znd gradient descent

Timestamp 25.28 / 1.43.32  

In [59]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [60]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [61]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [62]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.7786,  1.8568,  0.0227],
        [ 0.2765, -0.0965,  0.3487]], requires_grad=True)
tensor([-1.5746,  1.2085], requires_grad=True)


In [64]:
inputs @ w.t()+b

tensor([[253.6425,  29.9210],
        [325.1258,  40.1939],
        [403.2873,  32.5557],
        [260.5234,  38.1642],
        [300.9867,  35.4306]], grad_fn=<AddBackward0>)

In [65]:
def model(x):
    return x @ w.t()+b

In [66]:
preds=model(inputs)
print(preds)

tensor([[253.6425,  29.9210],
        [325.1258,  40.1939],
        [403.2873,  32.5557],
        [260.5234,  38.1642],
        [300.9867,  35.4306]], grad_fn=<AddBackward0>)


In [67]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [68]:
diff=preds-targets
diff * diff

tensor([[3.9063e+04, 1.6063e+03],
        [5.9597e+04, 3.6974e+03],
        [8.0819e+04, 1.0089e+04],
        [5.6893e+04, 1.3554e+00],
        [3.9199e+04, 6.9838e+03]], grad_fn=<MulBackward0>)

to get rid of negatives in diff, square numbers; * does a element wise multiplication